In [42]:
import pandas as pd
import numpy as np

data = pd.read_csv('dataset.csv',header = None)
data.columns = ['data']
data

,data
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [3]:
data['data'] = data['data'].apply(lambda x : str(x).replace("-", "" ))

In [4]:
data['data'] = data['data'].apply(lambda x : str(x).replace("_", "" ))

In [5]:
data['data'] = data['data'].apply(lambda x : str(x).replace(" ", "" ))

# Let's go the parts

In [6]:
data['hour'] = data['data'].apply(lambda x : x[0:4] )

In [7]:
def cleaning (col):
    col = col.replace("MM","M")
    if len(col) == 4 and col[3] != "M" :
        col = col[:-1]  # last character is removed     
    return col

In [8]:
data['hour'] = data['hour'].apply(cleaning)

In [9]:
def data_clean(col):
    
    col = col[1:]
    if col[0].isnumeric():
        col = col[1:]
        
    while True :
        index = 0
        for i in col:
            if i.isnumeric():
                col = col[index:]
                break
            index = index +1
        break
    return col

In [10]:
data['data'] = data['data'].apply(data_clean)

In [11]:
def cleaningData (col):
    
    col = col.replace("th","")
    col = col.replace("rd","")
    col = col.replace("st","")
    col = col.replace("nd","")
        
    return col

In [12]:
data['data'] = data['data'].apply(cleaningData)

# 

In [13]:
def date_column(col):
    index = col.find("2014") + 4
    col = col[:index]
    
    return col

In [14]:
data['date'] = data['data'].apply(date_column)

In [15]:
from dateutil import parser
def changing_type(col):
    col = parser.parse(col)
    
    return col

In [16]:
data['date'] = data['date'].apply(changing_type)

# 

In [17]:
def energy(col):
    index = col.find("2014") + 4
    col = col[index:]
    
    return col 

In [18]:
data['energy'] = data['data'].apply(energy)

In [19]:
data.drop('data',inplace = True , axis = 1)

In [20]:
def clean_energy(col):
    col = float(col[:-3])
    
    return col

In [21]:
data['energy'] = data['energy'].apply(clean_energy)

# 

In [22]:
import calendar
import datetime
from datetime import datetime

def weekdays(col):
    col = calendar.day_name[col.weekday()]
    
    return col

def month(col):
    num = col.month
    col = calendar.month_abbr[num]
    return col

def day(col):
    col = col.day
    return col

In [23]:
data['day'] = data['date'].apply(day)

In [24]:
data['weekday'] = data['date'].apply(weekdays)

In [25]:
data['month'] = data['date'].apply(month)

In [26]:
data.head(10) # result data

,hour,date,energy,day,weekday,month
0,3PM,2014-03-24,0.384,24,Monday,Mar
1,5AM,2014-08-15,1.201,15,Friday,Aug
2,8PM,2014-03-20,1.523,20,Thursday,Mar
3,6PM,2014-04-23,0.424,23,Wednesday,Apr
4,1AM,2014-12-19,0.209,19,Friday,Dec
5,5AM,2014-08-19,1.228,19,Tuesday,Aug
6,12PM,2014-07-07,1.296,7,Monday,Jul
7,7AM,2014-11-25,0.437,25,Tuesday,Nov
8,8AM,2014-08-14,0.523,14,Thursday,Aug
9,4PM,2014-01-25,2.052,25,Saturday,Jan


# 

In [27]:
data.describe() # The answer mean of energy - 0.7818 or 0.782 kWh

,energy,day
count,8760.000000,8760.000000
mean,0.781878,15.720548
std,0.759824,8.796749
min,0.048000,1.000000
25%,0.282000,8.000000
50%,0.523000,16.000000
75%,0.994000,23.000000
max,5.141000,31.000000


# 

In [28]:
data[data['month'] == 'Feb']['energy'].mean() # The answer mean of energy using in February - 0.833 kWh

0.8329151785714302

# 

In [29]:
weekdays = data.groupby(['weekday']).mean()
weekdays

,energy,day
weekday,,
Friday,0.753744,16.173077
Monday,0.725945,15.673077
Saturday,0.890764,15.442308
Sunday,0.898157,15.846154
Thursday,0.745458,15.769231
Tuesday,0.726058,15.500000
Wednesday,0.733943,15.641509


In [30]:
print(data.groupby(['weekday']).mean().idxmax())

energy    Sunday
day       Friday
dtype: object


# 

In [31]:
def hour_period(col):
    if col == '1AM' or col == '2AM' or col == '3AM' or col == '4AM' :
        col = '1'
    if col == '5AM' or col == '6AM' or col == '7AM' or col == '8AM' :
        col = '2'
    if col == '9AM' or col == '10AM' or col == '11AM' or col == '12AM' :
        col = '3'
    if col == '1PM' or col == '2PM' or col == '3PM' or col == '4PM' :
        col = '4'
    if col == '5PM' or col == '6PM' or col == '7PM' or col == '8PM' :
        col = '5'
    if col == '9PM' or col == '10PM' or col == '11PM' or col == '12PM' :
        col = '6'
    
    return col

In [32]:
data['period'] = data['hour'].apply(hour_period)

In [33]:
days = data.groupby(['month','day','period']).sum()
days

energy
month day period        
Apr   1   1        0.233
          2        1.055
          3        0.410
          4        0.525
          5        1.839
...                  ...
Sep   30  2        2.662
          3        0.917
          4        1.201
          5        4.153
          6        1.377

[2190 rows x 1 columns]

In [34]:
print(days.idxmax()) # month , day , period
print(days.max())

energy    (Jul, 20, 5)
dtype: object
energy    17.237
dtype: float64


# 

In [35]:
month_usage = pd.DataFrame(data.groupby('month')['energy'].sum())
month_usage = month_usage.reset_index()
month_usage

,month,energy
0,Apr,151.242
1,Aug,633.390
2,Dec,568.739
3,Feb,559.719
4,Jan,1042.660
5,Jul,1255.524
6,Jun,904.247
7,Mar,420.748
8,May,417.157
9,Nov,232.420


In [36]:
usage = month_usage['energy'].values

In [37]:
price = np.array([0.18 , 0.19 , 0.19 , 0.19 , 0.20 , 0.24 , 0.27 , 0.17 , 0.22 , 0.14 , 0.15 , 0.18 ])

In [38]:
cost = np.multiply(usage,price)
sum(cost)

1421.82096

# 

In [39]:
# The hourly plan
def cost_hour (col , energy):
    if col == '12AM' or col == '1AM' or col == '2AM' or col == '3AM' or col == '10PM' or col == '11PM' :
        energy = energy * 0.10
    if col == '4AM' or col == '4PM' :
        energy = energy * 0.15
    if col == '5AM' or col == '9AM' or col == '10AM' or col == '1PM' :
        energy = energy * 0.20
    if col == '6AM' or col == '7AM' or col == '2PM' or col == '6PM' or col == '7PM' :
        energy = energy * 0.24
    if col == '8AM' or col == '11AM' or col == '12PM':
        energy = energy * 0.26
    if col == '3PM' :
        energy = energy * 0.18
    if col == '5PM':
        energy = energy * 0.30
    if col == '8PM' :
        energy = energy * 0.12
    if col == '9PM' :
        energy = energy * 0.11

    return energy  

In [40]:
data['hour_cost'] = np.vectorize(cost_hour)(data['hour'], data['energy'])

Hourly_Flex = data['hour_cost'].sum()
Hourly_Flex

1369.3606300000001

In [41]:
# The No Flex plan
NoFlex = data['energy'].sum() * 0.21
NoFlex

1438.34271